# Logging best practices
The following code showcases how to use HCrystalBall with respect to the logging both from the library and wrapped libraries. Feel free to play around with the commented code to find a setup, which works best for your use-case.

There are two levels to customize 
 1. one for logging, available through `hcb_verbose` flag in the wrappers initialization (and available also through grid_search specification)
 1. one for warnings from statsmodels library available through `filter_statsmodels_warnings` function

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import logging
# import warnings
# warnings.simplefilter('ignore')
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
# from hcrystalball.utils import filter_statsmodels_warnings
# filter_statsmodels_warnings()

In [ ]:
# log_stats = logging.getLogger("py.warnings")
# log_stats.setLevel(logging.CRITICAL)

In [ ]:
# log_prophet = logging.getLogger("fbprophet")
# log_prophet.setLevel(logging.WARNING)

In [ ]:
from hcrystalball.utils import get_sales_data

df = get_sales_data(n_dates=100, 
                    n_assortments=2, 
                    n_states=2, 
                    n_stores=2)

In [ ]:
from hcrystalball.model_selection import ModelSelector

ms = ModelSelector(horizon=10, 
                   frequency='D', 
                   country_code_column='HolidayCode',                   
                  )

ms.create_gridsearch(sklearn_models=False,
                    n_splits = 2,
                    between_split_lag=None,
                    sklearn_models_optimize_for_horizon=False,
                    autosarimax_models=False,
                    prophet_models=False,
                    tbats_models=False,
                    exp_smooth_models=False,
                    average_ensembles=False,
                    stacking_ensembles=False,                    
                    exog_cols=['Open','Promo','SchoolHoliday','Promo2'],
                    hcb_verbose=False,
#                     holidays_days_before=2, 
#                     holidays_days_after=1, 
#                     holidays_bridge_days=True, 
                    )

In [ ]:
from hcrystalball.wrappers import get_sklearn_wrapper, ProphetWrapper, SarimaxWrapper, ExponentialSmoothingWrapper, ThetaWrapper
from sklearn.linear_model import LinearRegression

hcb_verbose = False

ms.add_model_to_gridsearch(get_sklearn_wrapper(LinearRegression, hcb_verbose=hcb_verbose))
ms.add_model_to_gridsearch(ProphetWrapper(hcb_verbose=hcb_verbose))
ms.add_model_to_gridsearch(SarimaxWrapper(init_with_autoarima=True, hcb_verbose=hcb_verbose))
ms.add_model_to_gridsearch(SarimaxWrapper(order=(1,0,1),hcb_verbose=hcb_verbose))
ms.add_model_to_gridsearch(ThetaWrapper(hcb_verbose=hcb_verbose))
ms.add_model_to_gridsearch(ExponentialSmoothingWrapper(hcb_verbose=hcb_verbose))

In [ ]:
df

In [ ]:
# from prefect.engine.executors import LocalDaskExecutor
ms.select_model(df=df,
                target_col_name='Sales',
                partition_columns=['Assortment', 'State','Store'],
#                 parallel_over_columns=['Assortment'],
#                 executor = LocalDaskExecutor(),                
               )

In [ ]:
# from prefect.engine.executors import LocalDaskExecutor
ms.select_model(df=df,
                target_col_name='Sales',
                partition_columns=['Assortment', 'State','Store'],
                parallel_over_columns=['Assortment'],
#                 executor = LocalDaskExecutor(),                
               )

In [ ]:
from prefect.executors import LocalDaskExecutor
ms.select_model(df=df,
                target_col_name='Sales',
                partition_columns=['Assortment', 'State','Store'],
                parallel_over_columns=['Assortment'],
                executor = LocalDaskExecutor(),                
               )